In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.interpolate
import numpy as np
import seaborn as sns
import env_flooding
import env_socioeconomic
import agent_generator
import agent_firm_basic
import agent_base
import random

In [2]:
# econs_dmg = pd.read_csv('data_model_inputs/flood_depth_damage_functions.tsv',
#                     sep='\t', comment='#', index_col=0, na_values=['N'])
# house_dmg = pd.read_csv('data_model_inputs/flood_depth_damage_functions2.tsv',
#                     sep='\t', comment='#', index_col=0, na_values=['N'])


In [3]:
bias_e = bias_t = None
steps_per_year = 4


extract = pd.read_csv('data/recovery_prices_asli22.tsv',
                      sep='\t', index_col=0, comment='#')

In [4]:
# ignore first line
prices = extract.loc[4:, 'discount']
error = extract.loc[4:, 'error']
prices_r =  np.insert(prices[1:].to_numpy() / prices[:-1].to_numpy(), 0, 1.)

# correct time index to model level (4 steps per year)
prices.index = prices.index * steps_per_year

# bias curve up/down based on error bars (Big caveat: error bars are just the SD)
if bias_e is not None:
    if abs(bias_e) <= 1:
        prices = prices - bias_e * error
    else:
        print('Warning: generate_asli_discount_curve can only handle -1 <= bias_e <= 1 ')

# bias curve length-wise
if bias_t is not None:
    prices.index = prices.index * bias_t


# think severe to light damaged places would converge to 0 at the same time??

In [5]:
import itertools
import model_experiment_utils as MEU

# model sets (for A)
params = {'A': [-1, 0, 1], 'B': [0.5, 0.75, 1.0, 1.25], "C": 'A'}
set_label = 'D'
constants = {"F": (1,2,3,4)}
blab_mSets = MEU.generate_model_sets(params, set_label, constants)
blab_fldT = MEU.generate_flood_timings(fld_events=[2],
                                       t_last_trailing=24,
                                       t_next_flood_lim=5,
                                       n_iters=3,
                                       t_first=0)
blab_fldS = MEU.generate_flood_scenarios([1,2,3], fld_events=[2], l_scenarios_per_cat=5)




In [8]:
# how to combine the big-ass list? Use itertools.product again?
blab_try = list(itertools.product(*[blab_mSets, blab_fldT, blab_fldS]))
# mSets = names, fldT = flood timings, fldS = flood scenarios

beeg_list = []
for munge in blab_try:
    mset_key, timing, scenario = munge
    mset = blab_mSets[mset_key]
    fld_time, sim_end = timing
    # need to make new name
    name = f"{mset_key}"
    # create into dict
    beeg_list.append((name, mset, fld_time, sim_end, scenario))

